In [ ]:
#import pandas and numpy for data analysis
import pandas as pd
import numpy as np

#import drive to set up Colab and Drive connection to access files in the Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Data Collection PLD1
- Import the first CSV for the first PLD file
  

**Question 1a)**
  - Please load the dataset and prepare it for analysis


In [ ]:
#import pld1 CSV from Google Drive folder CloudSort TakeHome 
pld1 = pd.read_csv('/content/drive/MyDrive/PLD1.csv') 
pld1.head()

,OrderCount,OriginZip,DestinationZip,Length,Width,Height,WeightOunces
0,1,12409,60197,12.0,9.0,1.0,6.0
1,2,49782,13144,9.0,8.0,6.0,24.0
2,1,42323,80920,12.0,9.0,1.0,6.0
3,1,17072,9499,12.0,9.0,5.0,67.0
4,1,49782,72068,9.0,6.0,4.0,11.0


# Data Exploration PLD1
- To understand the size of dataset, different data types we are dealing with it, and the number of missing values per column

In [ ]:
print(f'The total # rows in the dataset is :{len(pld1)}')
print('')
print(pld1.dtypes)
print('')
print(pld1.isnull().sum())

The total # rows in the dataset is :1580937

OrderCount          int64
OriginZip           int64
DestinationZip      int64
Length            float64
Width             float64
Height            float64
WeightOunces      float64
dtype: object

OrderCount        0
OriginZip         0
DestinationZip    0
Length            5
Width             5
Height            5
WeightOunces      6
dtype: int64


# Data Cleaning PLD1

There are a few ZipCodes for both 'OriginZip' and 'DestinationZip' that are only 4 digits, which seems to be an error as US Zipodes are generally 5 digits. The issue seems to be that the leading zero in the ZipCoped have disappeared when loading the CSV. This occurence is apparently common, as spreadsheets (CSV, excels) tend to interpret the column of ZipCode values as INTEGER, when what you really need is for them to be stored and formatted as STRING. By running the pld1.dtypes function, we can confirm that both columns for 'OriginZip' and 'DestinationZip' are datatype INTEGER.

- Below, I will run some tests to check if my above theory is correct about leading zero's disappearing is correct

In [ ]:
# Convert INTs to Strings for OriginZip and DestinationZip
pld1['OriginZip'] = pld1['OriginZip'].apply(str)
pld1['DestinationZip'] = pld1['DestinationZip'].apply(str)

#Find out how many Zip Codes in the columns 'OriginZip' and 'DestinationZip' have less than 5 characters
print('# of values in "OriginZip" Col with less than 5 digits is: ', 
      pld1['OriginZip'][pld1['OriginZip'].str.len() < 5].count())
print('')
print('# of values in "DestinationZip" Col with less than 5 digits is: ', 
      pld1['DestinationZip'][pld1['DestinationZip'].str.len() < 5].count())

# of values in "OriginZip" Col with less than 5 digits is:  88473

# of values in "DestinationZip" Col with less than 5 digits is:  127904


To validate my theory, I wrote code to check the number of ZipCodes that start with Zero below. The printed output says that we have no ZipCodes starting with Zero in the entire dataset. This is suspicious, and seems to validate my theory about the 0's having disappeared when being loaded due to data type issue.

In [ ]:
# check if theory makes sense, by checking how many ZipCodes start with 0
print('# of values in the "OriginZip" Col starting with Zero is:', 
      pld1['OriginZip'].loc[pld1['OriginZip'].str.startswith('0')].count())
print('# of values in the "DestinationZip" Col starting with Zero is:', 
      pld1['DestinationZip'].loc[pld1['DestinationZip'].str.startswith('0')].count())

# of values in the "OriginZip" Col starting with Zero is: 0
# of values in the "DestinationZip" Col starting with Zero is: 0


To adjust for the missing leading Zero's, I will manually impute a 0 in the beginning of the OriginZip and DestinationZip Columns using a function

In [ ]:
# define a function to add a '0' as the first character if the length of the string is less than 5
def add_zero_if_less_than_5(x):
    if len(x) < 5:
        return '0' + x
    else:
        return x

# apply the function to 'OriginZip' and 'DestinationZip' Columns
pld1['OriginZip'] = pld1['OriginZip'].apply(add_zero_if_less_than_5)
pld1['DestinationZip'] = pld1['DestinationZip'].apply(add_zero_if_less_than_5)

# to ensure that 0 have been successfully imputed, which they have, as now have thousand of records with zipcodes starting with 0
# the below numbers exactly match the previous outputted count of 4 digit zip code per Colum
print('# of values in the "OriginZip" Col starting with Zero is now :', 
      pld1['OriginZip'].loc[pld1['OriginZip'].str.startswith('0')].count())
print('# of values in the "DestinationZip" Col starting with Zero is now :', 
      pld1['DestinationZip'].loc[pld1['DestinationZip'].str.startswith('0')].count())

# of values in the "OriginZip" Col starting with Zero is now : 88473
# of values in the "DestinationZip" Col starting with Zero is now : 127904


**Question 1b)**
- In order to make comparisons across customers and time ranges, we normalize the package demand; create a feature of the AVG number of packages per day from a given OriginZip to DestinationZip (Assume this data is for the month of September 2022) 

In [ ]:
#import datetime library to work with dates
from datetime import datetime as dt 

In [ ]:
# First we need to get the number of days in September 2022
days_september = int((dt.strptime('2022/10/1', "%Y/%m/%d") - dt.strptime('2022/9/1', "%Y/%m/%d")).days)
print(f'The # of days in September is: {days_september}')

# Find all route combinations by using string concatenation to create routes for each shipment
pld1['Route'] = pld1['OriginZip'] + '_' \
               + pld1['DestinationZip']

# Find the Average # of packages shipped per day by dividing the sum of all packages shipped in a route with the total number days of september
df_avg_per_route = np.round(pd.DataFrame(pld1.groupby('Route')['OrderCount'].sum().sort_values(ascending=False) / days_september),decimals=3) #get the daily average per route and round decimals to 3
avg_packages_per_day = df_avg_per_route.rename(columns = {'OrderCount':'DailyAverage'}).reset_index() #rename column and reset index
pld1 = pd.merge(pld1, avg_packages_per_day, on = 'Route', how='inner') #add DailyAverage column to pld1 df using inner join on 'Route' column
print(avg_packages_per_day) #print the avg_packages_per_day distribution in desc order 
pld1.head() #print the pld1 df with the added feature

The # of days in September is: 30
              Route  DailyAverage
0       36039_71322        14.333
1       17072_27529         9.000
2       17072_06359         7.967
3       17072_12107         7.467
4       17072_91605         7.267
...             ...           ...
323933  52035_02337         0.033
323934  52035_02303         0.033
323935  52035_02215         0.033
323936  52035_02212         0.033
323937  17013_07456         0.033

[323938 rows x 2 columns]


,OrderCount,OriginZip,DestinationZip,Length,Width,Height,WeightOunces,Route,DailyAverage
0,1,12409,60197,12.0,9.0,1.0,6.0,12409_60197,0.267
1,2,12409,60197,12.0,9.0,1.0,8.0,12409_60197,0.267
2,1,12409,60197,10.0,6.0,1.0,2.0,12409_60197,0.267
3,1,12409,60197,9.0,10.0,8.0,46.0,12409_60197,0.267
4,1,12409,60197,20.0,8.0,4.0,39.0,12409_60197,0.267


**Question 2)** 
- In order to price shipments, we use the following guide to assigne each package a delivery Zone based on the delivery distance from OriginZip to DestinationZip
  - Zone 2 <-> 0-150 Miles
  - Zone 3 <-> 151-300 Miles
  - Zone 4 <-> 301-600 Miles
  - Zone 5 <-> 601-1000 Miles
  - Zone 6 <-> 1001-1400 Miles
  - Zone 7 <-> 1401-1800 Miles
  - Zone 8 <-> 1801 or more Miles

- Please provide the Delivery Zone for each shipment as a new feature and show you work/code.

In [ ]:
#install pgeocode library which is a python library for postal code geocoding and distance calculations
!pip3 install pgeocode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import pgeocode library which will be used to get the distance between zip codes (i.e. Routes) 
import pgeocode

In [ ]:
#create function to find the distance between ZipCodes (Routes)
def get_distance(x, y): 
  usa_zipcodes = pgeocode.GeoDistance('us') #define which country we are interested in
  distance_in_kms = usa_zipcodes.query_postal_code(x.values, y.values) #query_postal_code will get us the distance between ZipCodes in KM
  return distance_in_kms

# Apply get_distance function to OriginZip and DestinationZip Columns to get the distance between the columns and convert km to miles 
# Add Distance feature to pld1 DF 
pld1['Distance'] = np.round((get_distance(pld1['OriginZip'], pld1['DestinationZip']) * .6214), decimals=3)
pld1.head() #print the df

,OrderCount,OriginZip,DestinationZip,Length,Width,Height,WeightOunces,Route,DailyAverage,Distance
0,1,12409,60197,12.0,9.0,1.0,6.0,12409_60197,0.267,717.36
1,2,12409,60197,12.0,9.0,1.0,8.0,12409_60197,0.267,717.36
2,1,12409,60197,10.0,6.0,1.0,2.0,12409_60197,0.267,717.36
3,1,12409,60197,9.0,10.0,8.0,46.0,12409_60197,0.267,717.36
4,1,12409,60197,20.0,8.0,4.0,39.0,12409_60197,0.267,717.36


In [ ]:
# create a function to categorize shipments into Delivery Zones based on distance in Miles
def get_delivery_zone(distance):
  if distance >= 0 and distance <= 150:
    return 2 
  elif distance >= 151 and distance <= 300:
    return 3
  elif distance >= 301 and distance <= 600:
    return 4
  elif distance >= 601 and distance <= 1000:
    return 5
  elif distance >= 1001 and distance <= 1400:
    return 6
  elif distance >= 1401 and distance <= 1800:
    return 7
  elif distance >= 1801: 
    return 8
  else:
    return 'N/A'
    
#Apply get_delivery_zone to 'Distance' to categorize each delivery based on the distance
#Add 'DeliveryZone' column to pld1 dataframe
pld1['DeliveryZone'] = pld1['Distance'].apply(get_delivery_zone)

# print the Distance and DeliveryZone to make sure the output is accurate
pld1[['Distance', 'DeliveryZone']] 

,Distance,DeliveryZone
0,717.360,5
1,717.360,5
2,717.360,5
3,717.360,5
4,717.360,5
...,...,...
1580932,409.023,4
1580933,2540.913,8
1580934,1113.286,6
1580935,151.275,3


In [ ]:
!pip3 install pyzipcode

In [ ]:
from pyzipcode import ZipCodeDatabase

zcdb = ZipCodeDatabase()

In [ ]:
pld1['DestinationState'] = pld1['DestinationZip'].map(lambda x: zcdb[x].state if x in zcdb else 'N/A')

In [ ]:
pld1['OriginState'] = pld1['OriginZip'].map(lambda x: zcdb[x].state if x in zcdb else 'N/A')

In [ ]:
# print the final outcome of the pld1
pld1 = pld1[(pld1['DestinationState'] != 'N/A') | (pld1['OriginState'] != 'N/A')]
pld1.head()

,OrderCount,OriginZip,DestinationZip,Length,Width,Height,WeightOunces,Route,DailyAverage,Distance,DeliveryZone,DestinationState,OriginState
0,1,12409,60197,12.0,9.0,1.0,6.0,12409_60197,0.267,717.36,5,IL,NY
1,2,12409,60197,12.0,9.0,1.0,8.0,12409_60197,0.267,717.36,5,IL,NY
2,1,12409,60197,10.0,6.0,1.0,2.0,12409_60197,0.267,717.36,5,IL,NY
3,1,12409,60197,9.0,10.0,8.0,46.0,12409_60197,0.267,717.36,5,IL,NY
4,1,12409,60197,20.0,8.0,4.0,39.0,12409_60197,0.267,717.36,5,IL,NY


In [ ]:
def get_shipment_price(deliveryZone):
  if deliveryZone == 1:
    return 4.99
  elif deliveryZone == 2:
    return 4.99 + 1
  elif deliveryZone == 3:
    return 4.99 + 2
  elif deliveryZone == 4:
    return 4.99 + 3  
  elif deliveryZone == 5:
    return 4.99 + 4
  elif deliveryZone == 6:
    return 4.99 + 5
  elif deliveryZone == 7:
    return 4.99 + 6
  else:
    return 4.99 + 7

pld1['ShipmentPrice'] = pld1['DeliveryZone'].apply(get_shipment_price)
pld1.head()

,OrderCount,OriginZip,DestinationZip,Length,Width,Height,WeightOunces,Route,DailyAverage,Distance,DeliveryZone,DestinationState,OriginState,shipment_price,RevenuePerShipment,ShipmentPrice
0,1,12409,60197,12.0,9.0,1.0,6.0,12409_60197,0.267,717.36,5,IL,NY,8.99,8.99,8.99
1,2,12409,60197,12.0,9.0,1.0,8.0,12409_60197,0.267,717.36,5,IL,NY,8.99,17.98,8.99
2,1,12409,60197,10.0,6.0,1.0,2.0,12409_60197,0.267,717.36,5,IL,NY,8.99,8.99,8.99
3,1,12409,60197,9.0,10.0,8.0,46.0,12409_60197,0.267,717.36,5,IL,NY,8.99,8.99,8.99
4,1,12409,60197,20.0,8.0,4.0,39.0,12409_60197,0.267,717.36,5,IL,NY,8.99,8.99,8.99


In [ ]:
pld1['RevenuePerShipment'] = pld1['ShipmentPrice'] * pld1['OrderCount']

In [ ]:
pld1_final = pld1[['OriginZip', 'OriginState', 'DestinationZip', 'DestinationState','Route', 'OrderCount', 'DailyAverage',
      'Distance', 'DeliveryZone', 'ShipmentPrice', 'RevenuePerShipment']]
pld1_final.head()

,OriginZip,OriginState,DestinationZip,DestinationState,Route,OrderCount,DailyAverage,Distance,DeliveryZone,ShipmentPrice,RevenuePerShipment
0,12409,NY,60197,IL,12409_60197,1,0.267,717.36,5,8.99,8.99
1,12409,NY,60197,IL,12409_60197,2,0.267,717.36,5,8.99,17.98
2,12409,NY,60197,IL,12409_60197,1,0.267,717.36,5,8.99,8.99
3,12409,NY,60197,IL,12409_60197,1,0.267,717.36,5,8.99,8.99
4,12409,NY,60197,IL,12409_60197,1,0.267,717.36,5,8.99,8.99


In [ ]:
path = '/content/drive/MyDrive/pld1.csv' #save the final dataset to drive

with open(path, 'w', encoding = 'utf-8-sig') as df:
  pld1_final.to_csv(df)

# Data Collection PLD2
**Question 4 Step 1**
- Please load the next dataset 'PLD2.csv', from a different faux customer, and repeat steps 1 and 2. Assume all packages come from a single fulfillment center at 33602

In [ ]:
#import pld2
pld2 = pd.read_csv('/content/drive/MyDrive/PLD2.csv') #import pld2
pld2.head()

,shipmonth,shippingzipcode
0,2,77201
1,2,77201
2,2,77201
3,2,77201
4,2,77201


# Data Exploration PLD2
- To understand the size of dataset, different data types we are dealing with it, and the number of missing values per column

In [ ]:
print(f'The total # rows in the dataset is :{len(pld2)}')
print('')
print(pld2.dtypes)
print('')
print(pld2.isnull().sum())
print('')
print(pld2.head())

The total # rows in the dataset is :1331562

shipmonth          int64
shippingzipcode    int64
dtype: object

shipmonth          0
shippingzipcode    0
dtype: int64

   shipmonth  shippingzipcode
0          2            77201
1          2            77201
2          2            77201
3          2            77201
4          2            77201


# Data Cleaning PLD2

Considering the previously found issues in the PLD1 CSV, whereas, the leading Zero in ZipCodes disappeared, let's double check the PLD2 CSV to understand whether the 'shippingzipcode' column have similar issues.

In [ ]:
# Convert INT datatype to STRING for shippingzipcode 
pld2['shippingzipcode'] = pld2['shippingzipcode'].apply(str)

#find if/ how many Zip Codes are affected 
print('# of values in "shippingzipcode" Col with less than 5 digits is: ', 
      pld2['shippingzipcode'][pld2['shippingzipcode'].str.len() < 5].count())

# of values in "shippingzipcode" Col with less than 5 digits is:  106630


There seems to be a similar issue in the 2nd CSV, as we have 106630 values in the 'shippingzipcode' column where ZipCodes are only 4 digits

In [ ]:
# check if theory makes sense, by checking how many ZipCodes start with 0
print('# of values in the "shippingzipcode" Col starting with Zero is:', 
      pld2['shippingzipcode'].loc[pld2['shippingzipcode'].str.startswith('0')].count())

# of values in the "shippingzipcode" Col starting with Zero is: 0


Once again, we are able to confirm that there are no Zip Codes that start with Zero in the 'shippingzipcode' column.
- To adjust for the missing Zeros, I will manually impute a 0 in the beginning of the 'shippingzipcode' column using a function

In [ ]:
#convert shippingzipcode to string
pld2['shippingzipcode'] = pld2['shippingzipcode'].apply(str)

# define a function to add a '0' as the first character if the length of the string is less than 5
def add_zero_if_less_than_5(x):
    if len(x) < 5:
        return '0' + x
    else:
        return x

# apply the function to 'shippingzipcode'
pld2['shippingzipcode'] = pld2['shippingzipcode'].apply(add_zero_if_less_than_5)

#ensure that 0 have been successfully imputed, which they have, as now have thousand of records with zipcodes starting with 0
print('# of values in the "shippingzipcode" Col starting with Zero is now :',
      pld2['shippingzipcode'].loc[pld2['shippingzipcode'].str.startswith('0')].count())

# of values in the "shippingzipcode" Col starting with Zero is now : 106630


In [ ]:
#add originzipcode to pld2 dataframe
pld2['originzipcode'] = '33602'
pld2['originstate'] = 'FL'


# Get total # of days in February and March 2022 using datetime library
# We need total # of days to calculate the average # of packages shipped per day over a two-month period. 
# Total # of packages shipped during the entire two-month period is what matters for this calculation, not the # of days in each month separately.
# When dividing the total # of packages shipped by the total # of days in the period, you are calculating the average # of packages shipped per day
# Dividing by the total # of days in each month or dividing by 2 would not give you the correct average for the entire two-month period.

total_days = int((dt.strptime('2022/3/1', "%Y/%m/%d") - dt.strptime('2022/2/1', "%Y/%m/%d")).days) + \
                int((dt.strptime('2022/4/1', "%Y/%m/%d") - dt.strptime('2022/3/1', "%Y/%m/%d")).days)

print(f'The # of days in February + March is: {total_days}')

#Find all route combination by using string concatenation to create routes between 'originzipcode' and 'shippingzipcode'
pld2['Route'] = pld2['originzipcode'].apply(str) + '_' \
               + pld2['shippingzipcode'].apply(str)

The # of days in February + March is: 59


In [ ]:
pld2

,shipmonth,originzipcode,shippingzipcode,Route,DailyAverage,Distance,DeliveryZone,destinationstate,originstate
0,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL
1,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL
2,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL
3,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL
4,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL
...,...,...,...,...,...,...,...,...,...
1331557,2,33602,97709,33602_97709,0.017,2410.137,8,OR,FL
1331558,3,33602,04564,33602_04564,0.017,1317.336,6,ME,FL
1331559,3,33602,02539,33602_02539,0.017,1146.461,6,MA,FL
1331560,2,33602,99737,33602_99737,0.017,3693.508,8,AK,FL


In [ ]:
# get the average number of orders between routes for both months 
# assume that one row equals to one order as there is no OrderCount in PLD2
#group by route, get the Count per Route, divide by total # of days in the 2 month period
df1 = np.round(pd.DataFrame(pld2['Route'].value_counts()/total_days), decimals=3) 
avg_routes_per_day_pld2 = df1.reset_index().rename(columns = {'index':'Route', 'Route':'DailyAverage'}) #reset index and rename column
avg_routes_per_day_pld2.head()

,Route,DailyAverage
0,33602_18509,125.661
1,33602_05001,104.780
2,33602_51355,96.576
3,33602_97711,87.712
4,33602_97856,86.729


In [ ]:
#Add 'DailyAverage' column to PLD2 using inner join on the 'Route' column
pld2 = pd.merge(pld2, avg_routes_per_day_pld2, on = 'Route', how='inner') 

#reorder the columns in the pld2 dataframe
pld2 = pld2[['shipmonth', 'originzipcode', 'shippingzipcode', 'Route', 'DailyAverage']]

#print the final pld2 dataframe
pld2.head()

,shipmonth,originzipcode,shippingzipcode,Route,DailyAverage
0,2,33602,77201,33602_77201,66.322
1,2,33602,77201,33602_77201,66.322
2,2,33602,77201,33602_77201,66.322
3,2,33602,77201,33602_77201,66.322
4,2,33602,77201,33602_77201,66.322


**Question 4 Step 2**
- In order to price shipments, we use the following guide to assigne each package a delivery Zone based on the delivery distance from OriginZip to DestinationZip
  - Zone 2 <-> 0-150 Miles
  - Zone 3 <-> 151-300 Miles
  - Zone 4 <-> 301-600 Miles
  - Zone 5 <-> 601-1000 Miles
  - Zone 6 <-> 1001-1400 Miles
  - Zone 7 <-> 1401-1800 Miles
  - Zone 8 <-> 1801 or more Miles

- Please provide the Delivery Zone for each shipment as a new feature and show you work/code.

In [ ]:
#apply get_distance function to get the distance between zipcodes in PLD2

#add Distance feature to pld1 dataframe and convert km to miles 
pld2['Distance'] = np.round((get_distance(pld2['originzipcode'], pld2['shippingzipcode']) * .6214), decimals=3)
pld2.head()

,shipmonth,originzipcode,shippingzipcode,Route,DailyAverage,Distance
0,2,33602,77201,33602_77201,66.322,790.265
1,2,33602,77201,33602_77201,66.322,790.265
2,2,33602,77201,33602_77201,66.322,790.265
3,2,33602,77201,33602_77201,66.322,790.265
4,2,33602,77201,33602_77201,66.322,790.265


In [ ]:
# apply previously created get_delivery_zone function 
pld2['DeliveryZone'] = pld2['Distance'].apply(get_delivery_zone)

# print the Distance and DeliveryZone to make sure it looks good 
pld2[['Distance', 'DeliveryZone']] 

,Distance,DeliveryZone
0,790.265,5
1,790.265,5
2,790.265,5
3,790.265,5
4,790.265,5
...,...,...
1331557,2410.137,8
1331558,1317.336,6
1331559,1146.461,6
1331560,3693.508,8


In [ ]:
#print the final pld2 dataframe
pld2['destinationstate'] = pld2['shippingzipcode'].map(lambda x: zcdb[x].state if x in zcdb else 'N/A')
pld2.head()

,shipmonth,originzipcode,shippingzipcode,Route,DailyAverage,Distance,DeliveryZone,destinationstate
0,2,33602,77201,33602_77201,66.322,790.265,5,TX
1,2,33602,77201,33602_77201,66.322,790.265,5,TX
2,2,33602,77201,33602_77201,66.322,790.265,5,TX
3,2,33602,77201,33602_77201,66.322,790.265,5,TX
4,2,33602,77201,33602_77201,66.322,790.265,5,TX


In [ ]:
pld2 = pld2[pld2['destinationstate'] != 'FL']

In [ ]:
pld2['RouteState'] = pld2['originstate'] + '_' \
               + pld2['destinationstate']

In [ ]:
pld2

,shipmonth,originzipcode,shippingzipcode,Route,DailyAverage,Distance,DeliveryZone,destinationstate,originstate,RouteState
0,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL,FL_TX
1,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL,FL_TX
2,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL,FL_TX
3,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL,FL_TX
4,2,33602,77201,33602_77201,66.322,790.265,5,TX,FL,FL_TX
...,...,...,...,...,...,...,...,...,...,...
1331557,2,33602,97709,33602_97709,0.017,2410.137,8,OR,FL,FL_OR
1331558,3,33602,04564,33602_04564,0.017,1317.336,6,ME,FL,FL_ME
1331559,3,33602,02539,33602_02539,0.017,1146.461,6,MA,FL,FL_MA
1331560,2,33602,99737,33602_99737,0.017,3693.508,8,AK,FL,FL_AK


In [ ]:
path = '/content/drive/MyDrive/pld2.csv' #save the final dataset to drive

with open(path, 'w', encoding = 'utf-8-sig') as df:
  pld2[['originzipcode', 'shippingzipcode', 'Route',
        'destinationstate', 'originstate', 'RouteState']].to_csv(df)